In [1]:
import sys
import itertools
import shutil

sys.path.append("..")
from FIA.FIA import *
from file_handling import *

In [2]:
start = "../.." #"D:" # /mnt/d
paths = {"start":   [f"{start}/runs/FIA/"],
         "sub":     [ "Com20_single", "Com20_val", "Com8_equal_conc_comb", "Com8_grown_together", "Com20_repl", '20220620_P0003_S6-S22_FIA-TOF',
                     '20220726_FIA-TOF_Ex0005_msAG24-msAg77', '20220817_P0008_S13-S96_FIA-TOF', '20221221_P0008_S97-S156_FIA-TOF', '20230307_P0017_S1-S15_FIA-TOF',
                     '20230505_P0017_S16-S135_FI-TOF', '20230515_P0008_Plasma_Randomized', '20230515_P0008_Serum_Randomized', '20230516_P0008_Gewebe_Randomized',
                     '20230629_P0018_S01-S12_FI-TOF', '20230822_P0031_S1-S66_FI-TOF', '20230927_P0031_Decay_DHA_FI-TOF', '20230927_P0031_S001-S066_FI-TOF_1to100_wdh',
                     '20231121_P0032_FI-TOF', '20240206_P0038_S01-S12_FI-TOF', '20240216_P0032_plate10_SXX-SXX_FI-TOF', '20240327_TestAfterRelocation_FI-TOF'],
         "subsub":  ["merged"] }
path_combs = [os.path.normpath(os.path.join(*path)) for path in itertools.product(*paths.values())]
outpath = os.path.normpath(f"{start}/runs/FIA/all/merged")

In [3]:
for i, path in enumerate(tqdm(path_combs)):
    shutil.copy(os.path.join(path, "data_matrix.tsv"), os.path.join(outpath, f"data_matrix_{i}.tsv"))

  9%|▉         | 2/22 [00:04<00:45,  2.27s/it]


KeyboardInterrupt: 

In [5]:
combine_dc([os.path.join(outpath, file) for file in os.listdir(outpath) if file.endswith(".tsv")], outpath, target_format="feather", framework=pd)

  0%|          | 0/22 [00:00<?, ?it/s]

Loading: ../../runs/FIA/all/merged/data_matrix_16.tsv


  5%|▍         | 1/22 [00:00<00:17,  1.18it/s]

Loading: ../../runs/FIA/all/merged/data_matrix_11.tsv


  9%|▉         | 2/22 [00:07<01:23,  4.18s/it]

Loading: ../../runs/FIA/all/merged/data_matrix_5.tsv


 14%|█▎        | 3/22 [00:08<00:56,  2.95s/it]

Loading: ../../runs/FIA/all/merged/data_matrix_2.tsv


 14%|█▎        | 3/22 [00:38<04:00, 12.67s/it]


KeyboardInterrupt: 

In [ ]:
binned_dfs = []
for path in path_combs:
    print(f"Path: {path}")
    binned_df = pl.read_csv(os.path.join(path, "data_matrix.tsv"), separator="\t")
    binned_dfs.append( binned_df )

Path: /mnt/d/runs/FIA/Com20_single/merged
Path: /mnt/d/runs/FIA/Com20_val/merged
Path: /mnt/d/runs/FIA/Com8_equal_conc_comb/merged
Path: /mnt/d/runs/FIA/Com8_grown_together/merged
Path: /mnt/d/runs/FIA/Com20_repl/merged
Path: /mnt/d/runs/FIA/20220620_P0003_S6-S22_FIA-TOF/merged
Path: /mnt/d/runs/FIA/20220726_FIA-TOF_Ex0005_msAG24-msAg77/merged
Path: /mnt/d/runs/FIA/20220817_P0008_S13-S96_FIA-TOF/merged
Path: /mnt/d/runs/FIA/20221221_P0008_S97-S156_FIA-TOF/merged
Path: /mnt/d/runs/FIA/20230307_P0017_S1-S15_FIA-TOF/merged
Path: /mnt/d/runs/FIA/20230505_P0017_S16-S135_FI-TOF/merged
Path: /mnt/d/runs/FIA/20230515_P0008_Plasma_Randomized/merged
Path: /mnt/d/runs/FIA/20230515_P0008_Serum_Randomized/merged
Path: /mnt/d/runs/FIA/20230516_P0008_Gewebe_Randomized/merged
Path: /mnt/d/runs/FIA/20230629_P0018_S01-S12_FI-TOF/merged
Path: /mnt/d/runs/FIA/20230822_P0031_S1-S66_FI-TOF/merged
